In [ ]:
import polars as pl
import os
import re
import asyncio
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from tenacity import retry, stop_after_attempt, wait_exponential
from openai import AsyncOpenAI

# Патч для Jupyter/Kaggle
import nest_asyncio
nest_asyncio.apply()

# ==========================================
# 1. КОНФИГУРАЦИЯ DEEPSEEK
# ==========================================

API_KEY = os.getenv("NOVITA_API_KEY", "dljjfnsdkjfhsajfhsaksafasM") 
BASE_URL = "https://api.novita.ai/openai"
MODEL_NAME = "deepseek/deepseek-v3.2"

# Инициализация клиента
client = AsyncOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)

# Настройка задержки
DELAY_SECONDS = 20

print(f"Using Model: {MODEL_NAME}")

Using Model: deepseek/deepseek-v3.2


In [ ]:
# ==========================================
# 2. ПОДГОТОВКА ДАННЫХ
# ==========================================
mbti_df = pl.read_csv("/kaggle/input/mbti-type/mbti_1.csv") 
_, mbti_sample_pd = train_test_split(mbti_df.to_pandas(), test_size=800, stratify=mbti_df["type"].to_list(), random_state=69)
mbti_sample = pl.from_pandas(mbti_sample_pd)
print(f"MBTI Sample: {len(mbti_sample)}")

MBTI Sample: 800


In [ ]:
# ==========================================
# 3. ASYNC INFERENCE ENGINE (OPENAI CLIENT)
# ==========================================

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
async def get_llm_response(messages_list):
    """
    Асинхронный запрос через AsyncOpenAI (Novita).
    """
    try:
        response = await client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages_list,
            max_tokens=1024,
            temperature=0.5,
            response_format={"type": "json_object"} 
        )
        return response.choices[0].message.content
            
    except Exception as e:
        print(f"Request Failed: {e}")
        raise e

async def process_row(row, instruction_fn, parse_fn):
    """
    Обработка одной строки с разделением контекста
    """
    # 1. Формируем текст данных (посты)
    user_data_text = f"User Posts Data:\n{row['posts'][:3000]}"
    
    # 2. Формируем инструкцию
    instruction_text = instruction_fn()
    
    # 3. Создаем ИСТОРИЮ сообщений (контекст)
    messages = [
        {"role": "user", "content": user_data_text},
        {"role": "user", "content": instruction_text} 
    ]

    try:
        reply = await get_llm_response(messages)
        
        if reply is None:
            return None
            
        parsed = parse_fn(reply, row)
        return parsed
    except Exception as e:
        print(f"Failed to process row ID {row.get('id', 'unknown')}: {e}")
        return None

async def run_inference_with_delay(df, instruction_fn, parse_fn, max_rows=None):
    """
    Последовательная обработка
    """
    if max_rows and len(df) > max_rows:
        print(f"Limiting to first {max_rows} rows...")
        df = df.head(max_rows)

    results = []
    total_rows = len(df)
    
    print(f"Starting inference for {total_rows} rows.")
    
    for i, row in enumerate(df.iter_rows(named=True)):
        # 1. Запрос
        res = await process_row(row, instruction_fn, parse_fn)
        results.append(res)
        
        # 2. Лог
        valid_status = res is not None and res.get('type_pred') != "XXXX"
        print(f"[{i+1}/{total_rows}] Processed. Result valid: {valid_status}")

        # 3. Пауза
        if i < total_rows - 1:
            await asyncio.sleep(DELAY_SECONDS)
    
    valid_results = [r for r in results if r is not None]
    return pl.DataFrame(valid_results)

In [ ]:
# ==========================================
# 4. ПРОМПТЫ И МЕТРИКИ
# ==========================================

def get_mbti_instructions():
    """
    Возвращает ТОЛЬКО инструкцию и требования к формату.
    """
    return """Analyze the text data provided in the previous message to determine the author's MBTI type.

Evaluate based on these 4 dimensions:
1. (E) Extraversion vs (I) Introversion
2. (S) Sensing vs (N) Intuition
3. (T) Thinking vs (F) Feeling
4. (J) Judging vs (P) Perceiving

RETURN FORMAT:
You must return a valid JSON object ONLY:
{
  "reasoning": "Short analysis here...",
  "predicted_type": "INTJ"
}
"""

def mbti_parse(reply, row):
    try:
        # Очистка от маркдауна json
        clean_reply = reply.replace("```json", "").replace("```", "").strip()
        
        # 1. Пробуем парсить JSON
        json_match = re.search(r"\{.*\}", clean_reply, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            pred = data.get("predicted_type", "XXXX").upper()
            return {"type_true": row["type"], "type_pred": pred}
        
        # 2. Fallback regex (на случай сбоя JSON)
        strict_match = re.search(r"Type:?\s*\*?\*?([IE][NS][TF][JP])", reply, re.IGNORECASE)
        if strict_match:
            return {"type_true": row["type"], "type_pred": strict_match.group(1).upper()}

        simple_match = re.search(r"\b([IE][NS][TF][JP])\b", reply.upper())
        if simple_match:
             return {"type_true": row["type"], "type_pred": simple_match.group(1)}

        return {"type_true": row["type"], "type_pred": "XXXX"}
        
    except Exception as e:
        print(f"Parse Error: {e} | Reply snippet: {reply[:50]}...")
        return {"type_true": row["type"], "type_pred": "XXXX"}

all_metrics_data = []
def calculate_metrics(y_true, y_pred, task_name, dataset_name):
    valid_data = [(t, p) for t, p in zip(y_true, y_pred) if p != "XXXX" and len(p) == 4]
    
    if not valid_data:
        print(f"[{dataset_name}] No valid predictions to calculate metrics.")
        return

    y_true_clean = [x[0] for x in valid_data]
    y_pred_clean = [x[1] for x in valid_data]
    n = len(y_true_clean)

    print(f"\n--- Metrics for {task_name} (N={n}) ---")
    acc = accuracy_score(y_true_clean, y_pred_clean)
    _, _, f1_macro, _ = precision_recall_fscore_support(y_true_clean, y_pred_clean, average='macro', zero_division=0)

    print(f"Exact Match Accuracy: {acc:.2%}")
    print(f"Macro F1-Score:       {f1_macro:.2%}")

    axes = ["(I)E", "(N)S", "(T)F", "(J)P"]
    axis_scores = []
    total_letters_correct = 0
    for i in range(4):
        correct_count = sum(1 for t, p in zip(y_true_clean, y_pred_clean) if t[i] == p[i])
        axis_acc = correct_count / n
        axis_scores.append(axis_acc)
        total_letters_correct += correct_count
        print(f"Axis {axes[i]} Accuracy:      {axis_acc:.2%}")

    avg_letters = total_letters_correct / n
    print(f"Avg Letters Correct:  {avg_letters:.2f} / 4.00")
    
    all_metrics_data.append({
        "Dataset": dataset_name,
        "Exact_Acc": acc,
        "Macro_F1": f1_macro,
        "Avg_Letters": avg_letters
    })

    return all_metrics_data

In [ ]:
# ==========================================
# 5. MAIN
# ==========================================

async def main():
    if not mbti_sample.is_empty():
        print("\n=== Processing MBTI ===")
        res = await run_inference_with_delay(mbti_sample, get_mbti_instructions, mbti_parse)
        
        if not res.is_empty():
            res.write_csv("results_mbti_deepseek.csv")
            print("\nPreview of Results:")
            print(res.head())
            metrics = calculate_metrics(res["type_true"].to_list(), res["type_pred"].to_list(), "MBTI Type", "MBTI_Dataset")
        else:
            print("No results generated.")

In [6]:
await main()


=== Processing MBTI ===
Starting inference for 800 rows.
[1/800] Processed. Result valid: True
[2/800] Processed. Result valid: True
[3/800] Processed. Result valid: True
[4/800] Processed. Result valid: True
[5/800] Processed. Result valid: True
[6/800] Processed. Result valid: True
[7/800] Processed. Result valid: True
[8/800] Processed. Result valid: True
[9/800] Processed. Result valid: True
[10/800] Processed. Result valid: True
[11/800] Processed. Result valid: True
[12/800] Processed. Result valid: True
[13/800] Processed. Result valid: True
[14/800] Processed. Result valid: True
[15/800] Processed. Result valid: True
[16/800] Processed. Result valid: True
[17/800] Processed. Result valid: True
[18/800] Processed. Result valid: True
[19/800] Processed. Result valid: True
[20/800] Processed. Result valid: True
[21/800] Processed. Result valid: True
[22/800] Processed. Result valid: True
[23/800] Processed. Result valid: True
[24/800] Processed. Result valid: True
[25/800] Proces

In [7]:
res = pl.read_csv("results_mbti_deepseek.csv")
metrics = calculate_metrics(res["type_true"].to_list(), res["type_pred"].to_list(), "MBTI Type", "MBTI_Dataset")

mbti_df = mbti_sample.with_columns(
    res["type_pred"]
)
mbti_df = mbti_df.rename({"type_pred": "type_pred_deepseek"})
mbti_df = mbti_df.select(["type", "type_pred_deepseek", "posts"])
mbti_df.write_csv("mbti_deepseek_predictions.csv")


--- Metrics for MBTI Type (N=800) ---
Exact Match Accuracy: 52.50%
Macro F1-Score:       45.63%
Axis (I)E Accuracy:      85.50%
Axis (N)S Accuracy:      91.00%
Axis (T)F Accuracy:      76.62%
Axis (J)P Accuracy:      76.88%
Avg Letters Correct:  3.30 / 4.00
